### Explainer Experiment Demo

Below we show some examples on how you can use the **Experiment_Explainers** class. The general steps consist of:
1. prepare data
2. Initialize recommenders, explainers and metrics
3. Initialize experiment and run the experiment

You can set distribution=True to enable distribution plots while running the experiments. The plots will be saved in /experiment_plots.

When the experiment is completed, result table is saved in /experiment_plots.

In [1]:
import warnings
from cornac.datasets import goodreads
from cornac.experiment.experiment_explainers import Experiment_Explainers
from cornac.models import EMF, ALS, FMRec, MTER, EFM, NEMF
from cornac.explainer import Exp_EFM, Exp_LIMERS, Exp_MTER, Exp_ALS, Exp_PHI4MF, Exp_SU4EMF
from cornac.datasets.goodreads import prepare_data
from cornac.metrics_explainer import Metric_Exp_PSPNFNS as PSPN, Metric_Exp_PGF as PGF, Metric_Exp_MEP as MEP, Metric_Exp_EnDCG as EnDCG
from cornac.metrics_explainer import Metric_Exp_DIV as DIV, Metric_Exp_FPR as FPR, Metric_Exp_FA as FA, Metric_Exp_RA as RA

warnings.filterwarnings('ignore') 

/Users/yyz/miniconda3/envs/rose_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Experiment 1**: run PSPNFNS, FDIV and FPR on (FM, Exp_LIMERS) pair using data from goodreads_limers. This dataset contains explicit user and item features.

In [2]:
# prepare data
rs = prepare_data(data_name="goodreads_limers", test_size=0.2, dense=True, item=True, user=True, sample_size=1, seed=21)

# initialize recommenders, explainers and metrics
fm = FMRec()
limers = Exp_LIMERS(rec_model=fm, dataset=rs.train_set)
pspnfns = PSPN()
fdiv = DIV()
fpr = FPR()

# initialize experiment
models = [(fm, limers)]
metrics = [pspnfns, fdiv, fpr]
experiment = Experiment_Explainers(eval_method=rs, models=models, metrics=metrics,distribution=True, rec_k=10, feature_k=10, eval_train=True)
experiment.run()

INFO:cornac.experiment.experiment_explainers:Start training Recommender FMRec...


Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 0.94873
-- Epoch 2
Training MSE: 0.80318
-- Epoch 3
Training MSE: 0.76705
-- Epoch 4
Training MSE: 0.74580
-- Epoch 5
Training MSE: 0.72075
-- Epoch 6
Training MSE: 0.71087
-- Epoch 7
Training MSE: 0.69693
-- Epoch 8
Training MSE: 0.68820
-- Epoch 9
Training MSE: 0.67413
-- Epoch 10


INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'FMRec:Exp_LIMERS'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender FMRec creates recommendations


Training MSE: 0.66267


INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_LIMERS create explanation for all recommendations
Computing explanations: 100%|██████████| 1000/1000 [00:15<00:00, 64.22it/s]
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_PSPNFNS starts evaluation...
INFO:cornac.experiment.experiment_explainers:self.current_rec: FMRec, self.current_exp: Exp_LIMERS
Re-evaluate after features removal... : 100%|██████████| 1000/1000 [00:17<00:00, 58.02it/s]
INFO:cornac.experiment.experiment_explainers:Result: Probability of Necessity: 0.451; Probability of Sufficiency: 0.026; Harmonic Mean: 0.049165618448637316
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.5249212156596007
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_FPR starts evaluation...
ERROR:cornac.experiment.experiment_explainers:Metric Metric_Exp_FPR 

**Experiment 2**: run PSPN, FDIV and FPR on (EFM, Exp_EFM) and (MTER, Exp_MTER) pairs using data from goodreads. This dataset contains explicit review keywords.

In [6]:
# prepare data
rs = prepare_data(data_name="goodreads", test_size=0.2, dense=True, item=True, user=True, sample_size=1, seed=21)
ground_truth_good_reads = goodreads.load_sentiment()

# initialize recommenders and explainers
efm = EFM(max_iter=20, num_explicit_factors=50, num_latent_factors=50, num_most_cared_aspects=50, alpha=0.85,lambda_h=0.1, lambda_u=0.001, lambda_v=0.01, lambda_x=1.0, lambda_y= 0.1)
efm_exp = Exp_EFM(rec_model=efm, dataset=rs.train_set)
mter = MTER(max_iter=20, n_aspect_factors=8, n_item_factors=5, n_opinion_factors= 5, n_user_factors= 10, lambda_bpr=10, lambda_reg= 10, n_bpr_samples=1000, n_element_samples=50)
mter_exp = Exp_MTER(rec_model=mter, dataset=rs.train_set)

# initialize metrics
pspnfns = PSPN()
fdiv = DIV()
fpr = FPR()
fpr_with_input_as_groundtruth = FPR(ground_truth=ground_truth_good_reads)

# initialize experiment
models = [(efm, efm_exp), (mter, mter_exp)]
metrics = [fdiv, fpr_with_input_as_groundtruth, pspnfns]
experiment = Experiment_Explainers(eval_method=rs, models=models, metrics=metrics,rec_k=10, distribution=False, feature_k=10, eval_train=True)
experiment.run()

INFO:cornac.experiment.experiment_explainers:Start training Recommender EFM...
INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'EFM:Exp_EFM'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender EFM creates recommendations
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_EFM create explanation for all recommendations
Computing explanations: 100%|██████████| 1000/1000 [00:00<00:00, 7868.81it/s]
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.381047714381181
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_FPR starts evaluation...
Start evaluation... :  10%|█         | 720/7200 [00:00<00:04, 1368.36it/s]
INFO:cornac.experiment.experiment_explainers:Result: Feature Precision: 0.37784843667196605; Feature Recall: 0.06272186691193109; Harmonic Mean: 0.09959747000052623
INFO:

TypeError: list indices must be integers or slices, not str

**Experiment 3**: run FDIV, PGF, MEP and EnDCG on (ALS, Exp_ALS), (EMF, Exp_PHI4MF) and (NEMF, Exp_SU4EMF) pairs using data from goodreads. This dataset only contains user, item and rating info.

In [4]:
# prepare data
rs_dense = prepare_data(data_name="goodreads_uir_1000", test_size=0, dense=True, item=True, user=True, sample_size=1, seed=21)

# initialize recommenders and explainers
emf = EMF(k=10, max_iter=500, learning_rate=0.001, lambda_reg=0.1, explain_reg=0.01, verbose=True, seed=6, num_threads=6, early_stop=True)
nemf = NEMF(k=10, max_iter=500, learning_rate=0.001, lambda_reg=0.1, explain_reg=0.01, novel_reg=1, verbose=True, seed=6, num_threads=6, early_stop=True)
als = ALS(k=10, max_iter=500, lambda_reg=0.001, alpha=1, verbose=True, seed=6)
als_exp = Exp_ALS(rec_model=als, dataset=rs_dense.train_set)
emf_exp = Exp_PHI4MF(rec_model=emf, dataset=rs_dense.train_set)
nemf_exp = Exp_SU4EMF(rec_model=nemf, dataset=rs_dense.train_set)

# initialize metrics
fdiv = DIV()
pgf = PGF()
mep = MEP()
endcg = EnDCG()

# initialize experiment
models = [(als, als_exp), (emf, emf_exp), (nemf, nemf_exp)]
metrics = [fdiv, pgf, mep, endcg]
experiment = Experiment_Explainers(eval_method=rs_dense, models=models, metrics=metrics,distribution=False, rec_k=10, feature_k=10, eval_train=True)
experiment.run()

INFO:cornac.experiment.experiment_explainers:Start training Recommender ALS...
100%|██████████| 500/500 [00:01<00:00, 420.51it/s, loss=0.0284]
INFO:implicit:Final training loss 0.0284
INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'ALS:Exp_ALS'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender ALS creates recommendations
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_ALS create explanation for all recommendations
Computing explanations: 100%|██████████| 1000/1000 [00:02<00:00, 369.60it/s]
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.1549511511510353
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_PGF starts evaluation...
100%|██████████| 100/100 [00:00<00:00, 6405.77it/s]
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_PGF: 0.25898261626575325

Optimization finished!


Computing explanations:   2%|▏         | 21/1000 [00:42<23:17,  1.43s/it]  

Association rules generated


Computing explanations: 100%|██████████| 1000/1000 [00:47<00:00, 21.25it/s]
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.0020028028028030685
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_PGF starts evaluation...
100%|██████████| 100/100 [00:00<00:00, 2741.30it/s]
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_PGF: 0.08727997390553355
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_MEP starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_MEP: 0.992
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_EnDCG starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_EnDCG: 0.5637302146915871
INFO:cornac.experiment.experiment_explainers:Start training Recommender NEMF...


Start compute edge weight matrix...
Start compute novel matrix...
Matrix computation finished!


100%|██████████| 500/500 [00:00<00:00, 2547.60it/s, loss=6242.99]
INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'NEMF:Exp_SU4EMF'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Recommender NEMF creates recommendations
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer Exp_SU4EMF create explanation for all recommendations


Optimization finished!


Computing explanations: 100%|██████████| 1000/1000 [00:00<00:00, 1506.30it/s]
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_DIV starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Feature diversity: 0.39074534613965806
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_PGF starts evaluation...
100%|██████████| 100/100 [00:00<00:00, 3106.66it/s]
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_PGF: 0.24065170474350453
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_MEP starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_MEP: 0.996
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_EnDCG starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Metric_Exp_EnDCG: 0.5832008690101175
INFO:cornac.experiment.experiment_explainers:experiment data: [[0.1549511511510353, 0.25898261626575325, 'N/A', 'N/A', 1.219085693

**Experiment 4**: Run RA and FA on pairwise models. FA and RA performs comparison between two sets of explainers, thus the score is unavailable when only one explainer is passed.

In [5]:
# prepare data
rs = prepare_data(data_name="goodreads", test_size=0.2, dense=True, item=True, user=True, sample_size=1, seed=21)

# initialize recommenders and explainers
efm = EFM(max_iter=20, num_explicit_factors=50, num_latent_factors=50, num_most_cared_aspects=50, alpha=0.85,lambda_h=0.1, lambda_u=0.001, lambda_v=0.01, lambda_x=1.0, lambda_y= 0.1)
efm_exp = Exp_EFM(rec_model=efm, dataset=rs.train_set)
mter = MTER(max_iter=20, n_aspect_factors=8, n_item_factors=5, n_opinion_factors= 5, n_user_factors= 10, lambda_bpr=10, lambda_reg= 10, n_bpr_samples=1000, n_element_samples=50)
mter_exp = Exp_MTER(rec_model=mter, dataset=rs.train_set)

# initialize metrics
fa = FA()
ra = RA()
fdiv = DIV()

# initialize experiment
models = [[(efm, mter), (efm_exp, mter_exp)], (efm, efm_exp), (mter, mter_exp)]
metrics = [fa,ra, fdiv]
experiment = Experiment_Explainers(eval_method=rs, models=models, metrics=metrics,rec_k=10, feature_k=10, eval_train=True)
experiment.run()

INFO:cornac.experiment.experiment_explainers:Start training Recommender1 EFM...
INFO:cornac.experiment.experiment_explainers:Start training Recommender2 MTER...
INFO:cornac.experiment.experiment_explainers:*****Start evaluating model-explainer: 'EFM:Exp_EFM'vs'MTER:Exp_MTER'...
INFO:cornac.experiment.experiment_explainers:Step 1/3: Creates fake recommendations from dataset for common used
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer1 Exp_EFM create explanation for all recommendations
Computing explanations: 100%|██████████| 100/100 [00:00<00:00, 4503.08it/s]
INFO:cornac.experiment.experiment_explainers:Step 2/3: Explainer2 Exp_MTER create explanation for all recommendations
Computing explanations: 100%|██████████| 100/100 [00:00<00:00, 2598.03it/s]
INFO:cornac.experiment.experiment_explainers:Step 3/3: Metric Metric_Exp_FA starts evaluation...
INFO:cornac.experiment.experiment_explainers:Result: Average Metric_Exp_FA: 0.01
INFO:cornac.experiment.experiment_explainer